In [1]:
# Ml package Inports
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torchvision.datasets import ImageNet
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import dataset

In [2]:
# Helper package Imports
import numpy as np
import matplotlib.pyplot as plt
from zipfile import ZipFile
import os
import glob
from PIL import Image
from pathlib import Path
import splitfolders
import time
from tqdm.notebook import tqdm, trange
%matplotlib inline

In [3]:
# Extracting Dataset and renameing folder
if not Path("./Data/").is_dir() :
    with ZipFile('archive_5.zip',"r") as z:
        z.extractall(path="./Data/")
    old_name = str(os.listdir('./Data/')[0])
    os.renames('./Data/'+old_name, './Data/'+'Input')

In [4]:
# data directories
input_dir = './Data/Input/'
output_dir = './Data/Output/'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [5]:
# filename check
imgs = os.listdir(input_dir)
print(f'Total Images : {len(imgs)}')
print(f'Sample Image Name : {imgs[5]}')

Total Images : 10800
Sample Image Name : 000000010218.jpg


In [8]:
# change images
if os.path.exists(output_dir):
    for i in trange(len(imgs)):
        imgName = imgs[i]
        img = Image.open(input_dir+imgName).convert('LA')
        img.save(output_dir+imgName.replace('.jpg', '.png'))

In [9]:
# change one image
# img = Image.open(data_dir+imgs[0]).convert('LA')
# img.save(imgs[0].replace('.jpg', '.png'))
# img

In [10]:
# split data into train, validation and test data
def split_indices(n, val_pct, test_pct):
    # determine the size of the validation set, test set
    n_val = int(val_pct*n)
    n_test = int(test_pct*n + n_val)
    # print(n_val, n_test, n_train)
    # create random parmutation of 0 to n-1
    idxs = np.random.permutation(n)
    # pick data as train[start-val], validation[val-test] and test[test-end]
    return idxs[n_test:], idxs[:n_val], idxs[n_val:n_test]

In [128]:
?ImageFolder.make_dataset

In [49]:
# transformer
img_transform = transforms.Compose([
    transforms.CenterCrop(300),
#     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.ToTensor(),])

In [152]:
# DataSet
ImageData = ImageFolder('Data/',
                      transform = img_transform)

In [153]:
# split the dataset into training, validation and test set
train_indices, val_indices, test_indices = split_indices(len(dataset), 0.2, 0.1)
print(f"""
        Training Images   : {len(train_indices)}\n
        Validation Images : {len(val_indices)}\n
        Testing Images    : {len(test_indices)}\n
        Total Images      : {len(train_indices) + len(val_indices) + len(test_indices)}""")


        Training Images   : 5800

        Validation Images : 1656

        Testing Images    : 828

        Total Images      : 8284


In [154]:
# hyper parameter
batch_size=50

In [155]:
# training sampler and dataloader
train_sampler = SubsetRandomSampler(train_indices)    # takes samples w.r.t the indices
train_dl = DataLoader(dataset,
                      batch_size,
                      sampler = train_sampler)

In [156]:
# validation sampler and dataloader
val_sampler = SubsetRandomSampler(val_indices)        # takes samples w.r.t the indices
val_dl = DataLoader(dataset,
                    batch_size,
                    sampler = val_sampler)

In [157]:
# test sampler and dataloader
test_sampler = SubsetRandomSampler(test_indices)        # takes samples w.r.t the indices
test_dl = DataLoader(dataset,
                     batch_size,
                     sampler = test_sampler)

ValueError: too many values to unpack (expected 2)

In [ ]:
# create grayscale from color images

In [ ]:
# # get device function
def get_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [ ]:
# set device
device = get_device()
device

In [ ]:
# training data

In [ ]:
# validation data

In [ ]:
# sample

In [ ]:
# hyper parameters

In [ ]:
# model

In [ ]:
# fitting

In [ ]:
# output